##Connect to cloud

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##Trees

In [2]:
from google.colab import files
files.upload() 

Saving tree.py to tree.py


{'tree.py': b'import copy\r\nimport pandas as pd\r\nimport networkx as nx\r\nimport numpy as np\r\nimport matplotlib.pyplot as plt\r\nimport pickle\r\nimport os\r\n\r\nGOAL = "demand"\r\n# GOAL = "Goal"\r\n\r\n\r\nclass Node:\r\n    def __init__(self, data, choices=None, depth=0):\r\n        self.data = data\r\n        self.choices = choices\r\n        self.children = []\r\n        self.depth = depth\r\n\r\n    def __del__(self):\r\n        for child in self.children:\r\n            del child\r\n        del self.children\r\n\r\n    def add_children(self, children):\r\n        self.children = children\r\n\r\n\r\nclass Tree:\r\n    def __init__(self, records_df, limit=0, attributes=None, goal=GOAL, name=""):\r\n        """creates the tree based on a dictionary of attributes and options"""\r\n        self.limit = limit\r\n        self.goal = goal\r\n        self.name = name\r\n        if records_df is not None:\r\n            self.rows = len(records_df)\r\n            self.root = self.cre

##Constants

In [0]:
import pandas as pd
import math
from multiprocessing import Pool
from tree import *

In [0]:
PATH_TO_FOLDER = "/content/gdrive/My Drive/AI_project/"
INPUT_PATH = "Data/training_data_1.csv"
TEST_PATH = "Data/test_data_1.csv"
TRAINING_SET = "1"

GOAL = "demand"
TRAIN_LEVEL = 0.5
POOL_SIZE = 4
TREE = "Tree"
ENTROPY = "EntropyTree"
INFORMATION_GAIN = "InformationGainTree"
INFORMATION_RATIO = "InformationRatioTree"

BASIC_ATTRIBUTES = ["L1", "L2", "time"]
IGNORE_LIST = BASIC_ATTRIBUTES + ["Unnamed: 0", "Unnamed: 0.1", "cluster_id",
                                  "thunderstorm", "foggy", "humidity", "demand",
                                 "temperature"]

##Tree Creation

In [0]:
def tree_creation(type, records_df, limit=0, attributes=None, goal=GOAL, name=""):
    """This function creates the trees"""
#     if type == TREE:
#         return Tree(records_df, limit, attributes, goal, name)
    if type == ENTROPY:
        return EntropyTree(records_df, limit, attributes, goal, name)
    elif type == INFORMATION_GAIN:
        return InformationGainTree(records_df, limit, attributes, goal, name)
    elif type == INFORMATION_RATIO:
        return InformationRatioTree(records_df, limit, attributes, goal, name)
    else:
        return


def create_attributes_list(data):
    """Creates a list of attributes in order to build th trees"""
    attributes_list = []
    for column in data.columns:
        if column not in IGNORE_LIST:
            attributes_list.append(BASIC_ATTRIBUTES + [column])
    return attributes_list


def get_type(tree):
    """Gets the type of the tree"""
    return str(type(tree)).split('.')[1].split('\'')[0]


def create_trees(training_data, goal):
    p = Pool(POOL_SIZE)
    all_trees = []
    # every tree we want to create has to come in the format of
    # (type, df, limit, attributes, goal)
    attr_list = create_attributes_list(training_data)
    for lst in attr_list:
        print(lst[-1])
#         trees = [(TREE, training_data, 0, lst, goal, TREE + "_" + lst[-1] +
#                   "_" + TRAINING_SET)]
        trees = [(ENTROPY, training_data, 0, lst, goal,
                      ENTROPY + "_" + lst[-1] + "_" + TRAINING_SET)]
        trees.append((INFORMATION_GAIN, training_data, 0, lst, goal,
                      INFORMATION_GAIN + "_" + lst[-1] + "_" + TRAINING_SET))
        trees.append((INFORMATION_RATIO, training_data, 0, lst, goal,
                      INFORMATION_RATIO + "_" + lst[-1] + "_" + TRAINING_SET))
        res = p.starmap(tree_creation, trees)
        for t in res:
            t.save_tree(PATH_TO_FOLDER + "Trees/" + t.name + ".txt")
            all_trees.append(t)
    p.close()
    p.join()
    return all_trees


def export_training_and_test(training_data, test_data):
    """This functino saves the training data and the testing data"""
    training_data.to_csv("training_data.csv")
    test_data.to_csv("test_data.csv")
    
def load_data(path):
    """"""
    return pd.read_csv(path)

In [0]:
def additional_trees(training_data, goal):
    """"""
    p = Pool(POOL_SIZE)
    # every tree we want to create has to come in the format of
    # (type, df, limit, attributes, goal)
    all_trees = []
    new_basic = BASIC_ATTRIBUTES + ["weekday"]
    attr_list = [new_basic + ["month"]]
    attr_list.append(new_basic + ["clear_sky"])
    attr_list.append(new_basic + ["extreme_weather"])
#     attr_list.append(new_basic + ["clear_sky", "extreme_weather"])
    for lst in attr_list:
        filename = "_".join(list(set(lst) - set(BASIC_ATTRIBUTES)))
        print(filename)
#         trees = [(TREE, training_data, 0, lst, goal,
#                   TREE + "_" + filename + "_" + TRAINING_SET)]
        trees = [(ENTROPY, training_data, 0, lst, goal,
                      ENTROPY + "_" + filename + "_" + TRAINING_SET)]
        trees.append((INFORMATION_GAIN, training_data, 0, lst, goal,
                      INFORMATION_GAIN + "_" + filename + "_" + TRAINING_SET))
        trees.append((INFORMATION_RATIO, training_data, 0, lst, goal,
                      INFORMATION_RATIO + "_" + filename + "_" + TRAINING_SET))
        res = p.starmap(tree_creation, trees)
        for t in res:
            t.save_tree(PATH_TO_FOLDER + "Trees/" + t.name + ".txt")
            all_trees.append(t)
    p.close()
    p.join()
    return all_trees
    

In [0]:
    training_data = pd.read_csv(PATH_TO_FOLDER + INPUT_PATH)
    # create trees based on training data
    all_trees = create_trees(training_data, GOAL)
    print("finish normal trees")
    all_trees.extend(additional_trees(training_data, GOAL))

##Load all trees

In [0]:
from os import listdir
from os.path import isfile, join

def get_tree_files():
    onlyfiles = [f for f in listdir(PATH_TO_FOLDER + "Trees") if
             isfile(join(PATH_TO_FOLDER + "Trees", f)) and TRAINING_SET in f]
    return onlyfiles


def load_all_trees():
    files = get_tree_files()
    path = PATH_TO_FOLDER + "Trees/"
    trees = []
    for file in files:
        new_tree = Tree(None)
        new_tree.load_tree(join(PATH_TO_FOLDER + "Trees", file))
        trees.append(new_tree)
    return trees

In [9]:
all_trees = load_all_trees()
len(all_trees)

39

##Create file for NN

In [0]:
TEST_START = "Subsets/test_data_"

def create_file(test_data, all_trees, goal, part):
    """This function generates a file with the results of each tree and the
    actual result per line in the test data"""
    columns = [t.name for t in all_trees]
    columns.append(goal)
#     output = pd.DataFrame(columns=columns)
#     for i in range(len(test_data)):
#         print(str(i) + " out of " + str(len(test_data)), len(output))
#     while not test_data.empty:
#         test_data, row = test_data.iloc[1:], test_data.head(1)
#         print(part, len(test_data), len(output))
#         row_dict = {t.name: t.get_val(row) for t in all_trees}
#         row_dict[goal] = row[goal][0]
#         output = output.append(pd.DataFrame.from_dict([row_dict]))
#         del row

    output = pd.DataFrame()
    for t in all_trees:
        print(part, len(all_trees), all_trees.index(t))
        output[t.name] = test_data.apply(lambda row: t.get_val(row), axis=1)
    output[goal] = test_data[goal]


    if not output.empty:
        output.to_csv(PATH_TO_FOLDER + "Data/testing_by_tree_" + TRAINING_SET + "_" + str(part) + ".csv")
    del output
        
def change_numeric_to_format(data):
    for col in data.columns:
        if np.issubdtype(data[col].dtype, np.floating):
            data[col] = data[col].apply(lambda x: int(x))
        else:
            continue


def generate_file_for_part(part):
    test_data = pd.read_csv(PATH_TO_FOLDER + TEST_START + TRAINING_SET + "_" + part + ".csv")
    change_numeric_to_format(test_data)
    create_file(test_data, all_trees, GOAL, part)
    del test_data

In [0]:
start = 20
end = 199
test_range = []
for i in range(start,end+1):
    test_range.append("0" * (4 - len(str(i))) + str(i))

In [18]:
# TEST_RANGE = ["0001", "0002", "0003", "0004"]

p = Pool(30)

p.map(generate_file_for_part, test_range)

p.close()
p.join()

0029 39 0
0020 39 0
0026 39 0
0023 39 0
0065 39 0
0071 39 0
0059 39 0
0077 39 0
0068 39 0
0032 39 0
0074 39 0
0062 39 0
0056 39 0
0038 39 0
0047 39 0
0053 39 0
0035 39 0
0041 39 0
0050 39 0
0044 39 0
0029 39 1
0020 39 1
0068 39 1
0074 39 1
0059 39 1
0065 39 1
0026 39 1
0035 39 1
0071 39 1
0050 39 1
0044 39 1
0032 39 1
0023 39 1
0053 39 1
0077 39 1
0056 39 1
0062 39 1
0041 39 1
0038 39 1
0047 39 1
0029 39 2
0020 39 2
0074 39 2
0068 39 2
0059 39 2
0065 39 2
0026 39 2
0035 39 2
0050 39 2
0071 39 2
0044 39 2
0032 39 2
0053 39 2
0056 39 2
0023 39 2
0077 39 2
0062 39 2
0041 39 2
0038 39 2
0047 39 2
0029 39 3
0020 39 3
0074 39 3
0068 39 3
0059 39 3
0065 39 3
0026 39 3
0035 39 3
0050 39 3
0044 39 3
0071 39 3
0053 39 3
0056 39 3
0032 39 3
0029 39 4
0062 39 3
0077 39 3
0041 39 3
0038 39 3
0023 39 3
0047 39 3
0020 39 4
0074 39 4
0068 39 4
0065 39 4
0059 39 4
0026 39 4
0035 39 4
0044 39 4
0050 39 4
0071 39 4
0056 39 4
0053 39 4
0032 39 4
0062 39 4
0077 39 4
0041 39 4
0038 39 4
0023 39 4
0047 39 4
